In [3]:
import pandas as pd 
import numpy as np

BATCH1_2_XLSX_PATH = '../data/BCL-TKK-879- Dr. Korem- Final copy.xlsx'
BATCH3_XLSX_PATH = '../data/BCL-TKK-944 Korem- Oct 2023 copy.xlsx'
CYTOKINES = ['EGF', 'Flt-3L', 'G-CSF', 'IFNa2', 'IFNg', 'IL-10', 'IL-18', 'IL-1RA', 'IL-1a', 'IL-1b', 'IL-27', 'IL-4', 'IL-6', 'IL-8', 'MCP-1', 'RANTES', 'TGFa', 'TNFa', 'VEGF-A', 'sCD40L']

md = pd.read_csv('../data/md_124.csv',index_col=0)

# Load raw data
data_batches1_2 = pd.read_excel(BATCH1_2_XLSX_PATH, sheet_name='reformatted_data').set_index("Subject ID")
data_batch3 = pd.read_excel(BATCH3_XLSX_PATH, sheet_name='reformatted_data').set_index("Subject ID")
data = pd.concat([data_batches1_2,data_batch3])
data = data.loc[data.index.isin(md['ProMetVisit_1'])]

# Load LODs
LOD_batches1_2 = pd.read_excel(BATCH1_2_XLSX_PATH, sheet_name='limits_of_detection').T
LOD_batch3 = pd.read_excel(BATCH3_XLSX_PATH, sheet_name='limits_of_detection').T

for analyte in CYTOKINES:
    analyte_val_col = analyte + ' pg/ml'
    analyte_cv_col = analyte + ' CV%'
    for samp in data.index:
        if samp in data_batches1_2.index:
            analyte_LOD = LOD_batches1_2.loc[analyte_val_col,0]
        elif samp in data_batch3.index:
            analyte_LOD = LOD_batch3.loc[analyte_val_col,0]
        value = data.loc[samp, analyte_val_col]

        # Replace below LOD w/ NaN
        if '<' in str(value):
            data.loc[samp, analyte_val_col] = np.nan
            value = np.nan
        
        # Replace high CV w/ NaN
        curr_cv = data.loc[samp, analyte_cv_col]
        if curr_cv > 0.2:
            data.loc[samp, analyte_val_col] = np.nan
            value = np.nan

        # Impute low CV w/ LOD / sqrt(2)
        if value != value and curr_cv < 0.2: ## if the value is NaN
            data.loc[samp, analyte_val_col] = analyte_LOD / np.sqrt(2)
            
    data[analyte_val_col] = pd.to_numeric(data[analyte_val_col])
                   
luminex_by_batch = data[[c + ' pg/ml' for c in CYTOKINES]].rename(index={k:v for v,k in md['ProMetVisit_1'].to_dict().items()})
luminex_by_batch.to_csv('../data/luminex_lod_by_batch_124.csv')
luminex_by_batch


,EGF pg/ml,Flt-3L pg/ml,G-CSF pg/ml,IFNa2 pg/ml,IFNg pg/ml,IL-10 pg/ml,IL-18 pg/ml,IL-1RA pg/ml,IL-1a pg/ml,IL-1b pg/ml,IL-27 pg/ml,IL-4 pg/ml,IL-6 pg/ml,IL-8 pg/ml,MCP-1 pg/ml,RANTES pg/ml,TGFa pg/ml,TNFa pg/ml,VEGF-A pg/ml,sCD40L pg/ml
Subject ID,,,,,,,,,,,,,,,,,,,,
6300066P,94.42,4.760000,1408.000000,4.843681,5.520000,2.060000,196.50,7409.0,1297.00,183.620000,44.360000,0.490000,19.200000,3696.00,3.350000,3.330000,25.67,4.200000,49.03,76.56
2201103A,8.80,2.180000,121.660000,105.930000,4.400000,NaN,40.78,9176.0,3966.00,3.060000,49.630000,1.280000,1.720000,1433.00,6.520000,1.990000,15.30,2.330000,27.09,45.30
8400256K,138.78,1.280000,19.520000,11.340000,9.450000,1.510000,11.86,9696.0,1959.00,9.030000,16.072537,0.390000,2.880000,4147.00,10.180000,1.330000,2.90,1.393000,18.06,66.52
5100908P,143.12,0.643467,50.990000,44.700000,0.777817,1.410000,19.55,8284.0,430.46,1.430000,28.850000,0.640000,1.030000,197.70,11.680000,0.820244,9.98,1.393000,45.61,19.49
5101003K,32.56,1.280000,5.410000,NaN,0.777817,1.270000,41.69,8632.0,561.26,0.930000,34.520000,0.490000,0.339411,109.58,2.107178,0.820244,10.35,1.393000,8.94,16.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8500231T,76.33,4.750000,2.015254,2.156676,0.650000,0.438406,30.76,10735.0,199.60,0.487904,14.375481,0.340000,0.120208,39.98,1.796051,0.799031,4.60,1.421285,86.26,5.17
8300378D,157.20,0.304056,2.015254,2.156676,0.445477,0.920000,87.97,11898.0,187.38,1.100000,31.900000,0.400000,0.190000,272.88,9.400000,0.799031,3.24,1.421285,95.98,15.92
8300411I,103.72,4.720000,1030.000000,4.590000,3.320000,4.020000,190.59,11612.0,2335.00,53.570000,71.900000,0.300000,28.310000,9336.00,448.990000,125.670000,35.68,17.290000,3.89,34.71
